# 1. Import libraries

In [ ]:
# Pandas is the main library used for exploratory data analysis
# Pandas is built on top of numpy 
# Numpy allows fast advanced mathematical operations on large amount of data
import numpy as np
import pandas as pd

# Library forn saving a python object to file
import joblib 

# Libraries for further processing of the data
from sklearn.impute import SimpleImputer       # for assigning missing values

# Matplotlib is the main plotting library for python
# Seaborn is built on top of matplotlib and it's easier to use
# pandas also have same basic plotting capabilities which are again built on top of matplotlib
import matplotlib.pyplot as plt
# The next statement instructs matplotlib to plot the graphs inside the notebook
%matplotlib inline 
import seaborn as sns

# Another useful plotting library is plotly - it is used for interactive plots

# 2. Load and check data

In [ ]:
# Load the data - assuming its stored on your computer
#diabetes_df = pd.read_csv("diabetes.csv") # the suffix _df stands for dataframe
#diabetes_df = pd.read_csv("data/diabetes.csv") # if the CSV file is in folder data

# If you suspect missing values are coded in non-standard ways
#missing_values = ['?', '--', ' ', 'NA', 'N/A', '-'] # you can add more in the list
#diabetes_df = pd.read_csv("diabetes.csv", na_values=missing_values)

# If delimiter is not a comma
#diabetes_df = pd.read_csv("diabetes.csv", delimiter=';')

In [ ]:
# Load the data from the web
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
col_names = ['num_pregnancies', 
             'glucose', 
             'blood_pressure', 
             'skin_thickness', 
             'insulin', 
             'bmi', 
             'pedigree', 
             'age', 
             'outcome']
diabetes_df = pd.read_csv(url, header=None, names=col_names)

In [ ]:
diabetes_df.dtypes


In [ ]:
# SUPPOSE the type bmi was an "object" - meaning it's a string
# To convert any column that was loaded as a string to float use the astype(float) function
# Similarly, there are conversions for other data types
# Check documentation at 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
diabetes_df["bmi"] = diabetes_df["bmi"].astype(float)
diabetes_df.dtypes

# NOTE: If some of the values of column bmi were spaces
# Then trying to cast all the values to float will crash
# As a space cannot be converted to float
# Need to identify the rows with spaces
# And decide what to do with these rows
# Either delete these rows or replace them with something else (discussed under missing values)

# Suppose we dropping the rows with null values
# The following shows the code for how to do it
# Replace the spaces with null values (NaN) and then use dropna() to drop rows with Nan values
#diabetes_df["bmi"].replace(" ", np.nan, inplace=True) # modify the data in place - will return nothing
#diabetes_df.dropna(how='any', inplace=True) # any = drop a row with a Nan value in any column
# Check dropna() documentation at 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

!!! be sure to delete the duplicates. inplace = True. reset index. 

In [ ]:
# If you wanted to drop rows with duplicate values
print(diabetes_df.shape)

# Ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
diabetes_df.drop_duplicates(inplace=True) # will keep first copy

# You may want to reset the index
# REf: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
diabetes_df.reset_index(drop=True, inplace=True) # reset index to default integer index

# Check
print(diabetes_df.shape)

# 3. Handle missing values

a. Percentage of missing values encoded as NaN

In [ ]:
# Count of missing values in the whole dataframe by column
#diabetes_df.isnull().count() # does not work - gives the count of non-null values
#diabetes_df.isna().sum() # isna() is the same as isnull()

diabetes_df.isnull().sum()

In [ ]:
# Display percentage of missing values (NaN) per column
# isna() - returns a dataframe of all boolean values True/False
# Each value is tested  whether it's missing or not
# Boolean values are treated as numeric 0/1 when doing arithmetic operations
# So we can calculate the sum and mean of a column of boolean value
# Instead of calculating the sum and dividing by the number of rows, we can use the mean directly

percent_missing = round(diabetes_df.isna().mean() * 100, 4)
print(percent_missing)

b. What if missing values are not encoded as NaN? - be suspicious!!!


In [ ]:
# There are lots of zero values in some of the columns
diabetes_df[(diabetes_df['glucose'] == 0)]

In [ ]:
# Other datasets- missing values could be a single space e.g. " " one space inside the quotes
# This is not the case here
# Furthermore, you cannot check for spaces if the datatype of the column is float
# If there were spaces in the Glucose column, its datatype would be a string 
diabetes_df[(diabetes_df['glucose'] == " ")]

Change suspicious NaN data

In [ ]:
# Replace 0s in all columns except Pregnancies and Outcome

# Write code to get the column names
#columns = diabetes_df.columns.to_list()
#columns.remove('num_pregnancies')
#columns.remove('outcome')
# Or just specify the names of the columns
columns = ['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'pedigree', 'age']
print(columns)
for col in columns: # *in order to replace all values in DF not a single row. 
    #print(col)
    # Can replace one or more things at a time
    diabetes_df[col].replace(0, np.nan, inplace=True) # replace a 0 with NaN
    #diabetes_df[col].replace(" ", np.nan, inplace=True) # if some missing values were a space

# Check - the percentages of missing values should show up in each column   
print(round(diabetes_df.isna().mean() * 100, 4))

c. If you wanted to drop all rows with missing values (NaN) - simply dropping the record is not always an answer.

In [ ]:
# If you are dropping NaN or NA values, you can simply use the dropna() function

# Ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
# Examples: https://www.w3resource.com/pandas/dataframe/dataframe-dropna.php

# There are several ways to use the dropna() function

# Alternative 1. Drop the rows where at least one element is missing
diabetes_df.dropna(inplace=True)

# Alternative 2. Drop the rows where all elements are missing.
diabetes_df.dropna(how='all', inplace=True)

# Alternative 3. Keep only the rows with at least 2 non-NA values
diabetes_df.dropna(thresh=2, inplace=True)

# Alternative 4. Define in which columns to look for missing values
# You can check in one or more columns
# This is just an example to demonstrate
diabetes_df.dropna(subset=['glucose', 'blood_pressure']) 

# Please note that the above commands will only work 
# if the missing values are NaN (not a number) or NA (not available)
# They will not work if the missing values are 0s.

# Check - there should be zero percent missing values in all columns
#print(round(diabetes_df.isna().mean() * 100, 4))

# It'a a good idea to check the number of rows and columns left after dropping rows
#print(diabetes_df.shape)

d. If you wanted to replace all missing values (NaN) with median values


In [ ]:
# Replace NaN with median value in all columns
columns = ['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'pedigree', 'age']

# If you want to use SKlearn's SimpleInputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')

for col in columns:
    print(col)
    # Option 1. Using SimpleImputer to replace all missing values in column with median value
    diabetes_df[col] = imp_median.fit_transform(diabetes_df[col].values.reshape(-1, 1))
    # Option2. (samle function as above) You can also use pandas to do the replacement (also called imputation)
#     diabetes_df[col].fillna(diabetes_df[col].median(), inplace=True)

# Check - there should be zero percent missing values in all columns
print(round(diabetes_df.isna().mean() * 100, 4))

# 6. Handle outliers

1. visualize outliers

In [ ]:
# Example 1 - age 
# Ref: https://seaborn.pydata.org/generated/seaborn.boxplot.html

sns.boxplot(x=diabetes_df["age"]) # if used y=x=diabetes_df["age"] - plot displayed vertically

In [ ]:
# Example 2 - number of pregnancies
sns.boxplot(x=diabetes_df["num_pregnancies"])

In [ ]:
# Example 3 - subplots

fig, ax = plt.subplots(2,4, figsize=(16,6))  # 2 rows and 4 columns

# Row 0
ax1 = sns.boxplot(y=diabetes_df["num_pregnancies"], ax=ax[0,0]) # row 0, col 0
ax2 = sns.boxplot(y=diabetes_df["glucose"], ax=ax[0,1]) 
ax3 = sns.boxplot(y=diabetes_df["blood_pressure"], ax=ax[0,2])
ax4 = sns.boxplot(y=diabetes_df["skin_thickness"], ax=ax[0,3])

# Row 1
ax5 = sns.boxplot(y=diabetes_df["insulin"], ax=ax[1,0])
ax6 = sns.boxplot(y=diabetes_df["bmi"], ax=ax[1,1])
ax7 = sns.boxplot(y=diabetes_df["pedigree"], ax=ax[1,2])
ax8 = sns.boxplot(y=diabetes_df["age"], ax=ax[1,3])

c. Percentage outliers


In [ ]:
# Finding the percentages of outlier per columns
# Note: Outliers only apply to continuous numerical values - not to strings or numerical categories!
# So, be careful what you include in the list of column names

# Add Pregnancies to the list
columns = ['num_pregnancies', 'glucose', 'blood_pressure', 
           'skin_thickness', 'insulin', 'bmi', 'pedigree', 'age']
print(columns)
print()

for col in columns:
    percentile_25 = diabetes_df[col].quantile(0.25)
    percentile_75 = diabetes_df[col].quantile(0.75)
    iqr = percentile_75 - percentile_25 # Inter Quartile Range
    total = len(diabetes_df[col])
    #total = diabetes_df[col].shape[0] # also works

    # Determine the upper and lower boundaries for outliers
    # The lower and upper boundaries are less than the bottom whisker or greater than the top whisker
    #cut_off = iqr * 3    # three times IQR - for detecting extreme outliers
    cut_off = iqr * 1.5  # normally use 1.5 times IQR
    lower, upper = percentile_25 - cut_off, percentile_75 + cut_off
    print(f"col: {col}, lower: {lower}, upper: {upper}")

    # Count all values that are less than the lower boundary OR those that are greater than the upper boundary
    #num_outliers = diabetes_df[(diabetes_df[col] < lower) | (diabetes_df[col] > upper)].count() 
    # returns a list of things
    num_outliers = len(diabetes_df[(diabetes_df[col] < lower) | (diabetes_df[col] > upper)])
    
    pc_outliers = round(num_outliers*100/total,2)
    
    print(f"Num outliers: {num_outliers}, total rows: {total}, percent: {pc_outliers}")
    print()

d. Replace outliers with median


In [ ]:
# Using Inter Quartile Range (IQR) to detect outliers - for non-normal (non-Gaussian) distributions

# Add Pregnancies to the list
columns = ['num_pregnancies', 'glucose', 'blood_pressure', 
           'skin_thickness', 'insulin', 'bmi', 'pedigree', 'age']

for col in columns:
    median_val = diabetes_df[col].quantile(0.50)
    percentile_25 = diabetes_df[col].quantile(0.25)
    percentile_75 = diabetes_df[col].quantile(0.75)
    iqr = percentile_75 - percentile_25 # Inter Quartile Range
    
    # Determine the upper and lower boundaries for outliers
    # The lower and upper boundaries are less than the bottom whisker or greater than the top whisker
    #cut_off = iqr * 3    # three times IQR - for detecting extreme outliers
    cut_off = iqr * 1.5  # normally use 1.5 times IQR
    lower, upper = percentile_25 - cut_off, percentile_75 + cut_off
    print(f"col: {col}, lower: {lower}, upper: {upper},  median_val: {median_val}")
    
    # Replace all outliers in column with median value
    # Replace all values that are less than the lower boundary OR those that are greater than the upper boundary
    diabetes_df[col] = np.where((diabetes_df[col] < lower) | (diabetes_df[col] > upper), median_val, diabetes_df[col])
    # **where: just like where method in SQL


#diabetes_df.describe()
diabetes_df.describe().T  # T means transpose - will transpose rows to column

# 7. Row manipulation

a. Drop rows with certain values


In [ ]:
# Drop rows where blood_pressure is greater than 120 - considering it as an outlier
# Ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

# Specify the condition
condition = diabetes_df['blood_pressure'] > 120
# If you wanted to check if there are rows with null values in this column, then your condition would be:
#condition = diabetes_df['blood_pressure'] = " " # Note that there is a single space between the quotes
# You can also specify more complex conditions e.g.
#condition = diabetes_df['blood_pressure'] > 120 and diabetes_df['bmi'] == 0 
# note two equal signs to test for equality
#condition = diabetes_df['blood_pressure'] > 120 and diabetes_df['bmi'] != 0 
# != means not equal

# Display the rows
diabetes_df[condition] 

# The above statement will filter the rows according to the condition specified 
# and the output atomatrically displayed
# However, for the output to be automatically display, 
# the statement should be the last one in the cell
# If you want to execute some other statements, place them in the next cell

In [ ]:
# Then drop the rows according to the previously defined condition (if you decide to)
diabetes_df.drop(diabetes_df[condition].index, axis=0, inplace=True) # axis=0 drop rows, 1 is for columns

# Check
condition = diabetes_df['blood_pressure'] > 110 # set this lower than 120 so you can see some rows
diabetes_df[condition] # the output from this statement will display as it's the last statement

a. Rename columns


In [ ]:
# SUPPOSE you wanted to change the names of one or more columns 
# This is just sample code to illustrate how to do it
# The code is commented out to prevent it from running (executing)

# Define a dictionary mapping current column names to new names
# Can change the name of one or more columns
new_column_names = {
   'bmi': 'BMI', 
   'pedigree': 'diabetes_pedigree_function', 
}

# Rename
#diabetes_df = diabetes_df.rename(columns=new_column_names)

# Check
#diabetes_df.head()

c. Drop a column


In [ ]:
# ANOTHER ILLUSTRATION
# Suppose you want to drop the column "blood_pressure"
# Again, this is just sample code to illustrate how to do it
# The code is commented out to prevent it from running (executing)
diabetes_df.drop(['blood_pressure'], axis=1, inplace=True)

Renaming values (if required)


In [ ]:
# ANOTHER ILLUSTRATION
# Suppose you want to rename the outcomes 0/1 to "Yes" and "no"
# Again, this is just sample code to illustrate how to do it
# The code is commented out to prevent it from running (executing)

#diabetes_df["outcome"] = diabetes_df["outcome"].replace({1:"Yes",0:"No"})

# Check
#diabetes_df.head()

# 10. View cleaned data and pickle

In [ ]:
# You might want to do a final check on the data
diabetes_df.head() 

# You could also save the cleaned data as a CSV file
# However, when reloading the file again, floats might have changed back to strings (need to change again)
#diabetes_df.to_csv('diabetes-cleaned.csv', index=False) # Save in current folder
#diabetes_df.to_csv('data/diabetes-cleaned.csv', index=False) # save in data folder - make sure it exists
# If you saved the cleaned data as a CSV file, you can view its contents using Excel

# Note: If the last statement is saving to CSV, then you wont see the result of the head() function.
# Either, put the statement diabetes_df.head() last in the cell
# Or, use print(diabetes_df.head()), but then you will lose the nice formatting!

In [ ]:
# Save the object as a pickle in a file 
joblib.dump(diabetes_df, 'diabetes-cleaned.pkl') 
  
# In the other notebook    
# Load the model from the file 
#diabetes_df = joblib.load('diabetes-cleaned.pkl') 